<a href="https://colab.research.google.com/github/isurusilva/Video_Classifier/blob/main/video_classifier_working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Classifier Using CNN and RNN
#!dir

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('/content/drive/MyDrive/Video_classifier_example/Datasets/train')

label_types = os.listdir('/content/drive/MyDrive/Video_classifier_example/Datasets/train')
print (label_types)

['fight', 'no fight']


# Preparing Training Data

In [3]:
rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/Video_classifier_example/Datasets/train' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/Video_classifier_example/Datasets/train' + '/' +item) + '/' + room))

# Build a dataframe
train_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())


     tag                                         video_name
0  fight  /content/drive/MyDrive/Video_classifier_exampl...
1  fight  /content/drive/MyDrive/Video_classifier_exampl...
2  fight  /content/drive/MyDrive/Video_classifier_exampl...
3  fight  /content/drive/MyDrive/Video_classifier_exampl...
4  fight  /content/drive/MyDrive/Video_classifier_exampl...
          tag                                         video_name
555  no fight  /content/drive/MyDrive/Video_classifier_exampl...
556  no fight  /content/drive/MyDrive/Video_classifier_exampl...
557  no fight  /content/drive/MyDrive/Video_classifier_exampl...
558  no fight  /content/drive/MyDrive/Video_classifier_exampl...
559  no fight  /content/drive/MyDrive/Video_classifier_exampl...


In [4]:
df = train_df.loc[:,['video_name','tag']]
df
df.to_csv('train.csv')

# Preparing Test Data

In [5]:
dataset_path = os.listdir('/content/drive/MyDrive/Video_classifier_example/Datasets/test')
print(dataset_path)

room_types = os.listdir('/content/drive/MyDrive/Video_classifier_example/Datasets/test')
print("Types of activities found: ", len(dataset_path))

rooms = []

for item in dataset_path:
 # Get all the file names
 all_rooms = os.listdir('/content/drive/MyDrive/Video_classifier_example/Datasets/test' + '/' +item)

 # Add them to the list
 for room in all_rooms:
    rooms.append((item, str('/content/drive/MyDrive/Video_classifier_example/Datasets/test' + '/' +item) + '/' + room))

# Build a dataframe
test_df = pd.DataFrame(data=rooms, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df
df.to_csv('test.csv')

['fight', 'no fight']
Types of activities found:  2
     tag                                         video_name
0  fight  /content/drive/MyDrive/Video_classifier_exampl...
1  fight  /content/drive/MyDrive/Video_classifier_exampl...
2  fight  /content/drive/MyDrive/Video_classifier_exampl...
3  fight  /content/drive/MyDrive/Video_classifier_exampl...
4  fight  /content/drive/MyDrive/Video_classifier_exampl...
          tag                                         video_name
144  no fight  /content/drive/MyDrive/Video_classifier_exampl...
145  no fight  /content/drive/MyDrive/Video_classifier_exampl...
146  no fight  /content/drive/MyDrive/Video_classifier_exampl...
147  no fight  /content/drive/MyDrive/Video_classifier_exampl...
148  no fight  /content/drive/MyDrive/Video_classifier_exampl...


In [6]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to /tmp/pip-req-build-1nswajeo
  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/docs /tmp/pip-req-build-1nswajeo
  Resolved https://github.com/tensorflow/docs to commit 2b700605aaf42a346624aaff5c84879999d4c407
  Preparing metadata (setup.py) ... done
  Created wheel for tensorflow-docs: filename=tensorflow_docs-2023.9.4.19335-py3-none-any.whl size=184104 sha256=e66d2ea02aa97788fca446f127b605a5c0cfb9622f8b5b277b449019925ac75f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5enpn2z3/wheels/86/0f/1e/3b62293c8ffd0fd5a49508e6871cdb7554abe9c62afd35ec53
Successfully built tensorflow-docs


In [7]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [8]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

# Data preparation

In [9]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")


train_df.sample(10)

Total videos for training: 560
Total videos for testing: 149


,Unnamed: 0,video_name,tag
322,322,/content/drive/MyDrive/Video_classifier_exampl...,no fight
535,535,/content/drive/MyDrive/Video_classifier_exampl...,no fight
198,198,/content/drive/MyDrive/Video_classifier_exampl...,fight
423,423,/content/drive/MyDrive/Video_classifier_exampl...,no fight
195,195,/content/drive/MyDrive/Video_classifier_exampl...,fight
374,374,/content/drive/MyDrive/Video_classifier_exampl...,no fight
289,289,/content/drive/MyDrive/Video_classifier_exampl...,no fight
93,93,/content/drive/MyDrive/Video_classifier_exampl...,fight
264,264,/content/drive/MyDrive/Video_classifier_exampl...,fight
529,529,/content/drive/MyDrive/Video_classifier_exampl...,no fight


# Feed the videos to a network:


In [10]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

   ### Feature Extraction

In [11]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 [==============================] - 1s 0us/step


### Label Encoding
StringLookup layer encode the class labels as integers.

In [12]:
label_processor = keras.layers.StringLookup(num_oov_indices=0, vocabulary=np.unique(train_df["tag"]))
print(label_processor.get_vocabulary())

labels = train_df["tag"].values
labels = label_processor(labels[..., None]).numpy()
labels

['fight', 'no fight']


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

Finally, we can put all the pieces together to create our data processing utility.

In [ ]:
#print(train_data[0].shape)
#train_data[0]

In [13]:
#Define hyperparameters

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [14]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()

    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["tag"].values

    #convert classlabels to label encoding
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")

# MAX_SEQ_LENGTH = 20, NUM_FEATURES = 2048. We have defined this above under hyper parameters

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 62ms/step
Frame features in train set: (560, 20, 2048)
Frame masks in train set: (560, 20)
train_labels in train set: (560, 1)
test_labels in train set: (149, 1)


# The sequence model
Now, we can feed this data to a sequence model consisting of recurrent layers like GRU.

In [15]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model

EPOCHS = 30
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/30
12/13 [==========================>...] - ETA: 0s - loss: 0.6151 - accuracy: 0.7161
Epoch 1: val_loss improved from inf to 1.33909, saving model to ./tmp/video_classifier
13/13 [==============================] - 15s 238ms/step - loss: 0.6143 - accuracy: 0.7143 - val_loss: 1.3391 - val_accuracy: 0.0000e+00
Epoch 2/30
13/13 [==============================] - ETA: 0s - loss: 0.5168 - accuracy: 0.7194
Epoch 2: val_loss did not improve from 1.33909
13/13 [==============================] - 0s 21ms/step - loss: 0.5168 - accuracy: 0.7194 - val_loss: 1.8159 - val_accuracy: 0.0000e+00
Epoch 3/30
13/13 [==============================] - ETA: 0s - loss: 0.4625 - accuracy: 0.7194
Epoch 3: val_loss did not improve from 1.33909
13/13 [==============================] - 0s 21ms/step - loss: 0.4625 - accuracy: 0.7194 - val_loss: 1.5799 - val_accuracy: 0.0000e+00
Epoch 4/30
13/13 [==============================] - ETA: 0s - loss: 0.3988 - accuracy: 0.7500
Epoch 4: val_loss did not improve from 

# Inference

In [16]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames

# test_video = np.random.choice(test_df["video_name"].values.tolist())
test_video = '/content/drive/MyDrive/Video_classifier_example/Datasets/test/no fight/no492_xvid.avi'
print(f"Test video path: {test_video}")

test_frames = sequence_prediction(test_video)


Test video path: /content/drive/MyDrive/Video_classifier_example/Datasets/test/no fight/no492_xvid.avi
1/1 [==============================] - 2s 2s/step
  fight: 76.18%
  no fight: 23.82%


In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" width="520" height="440" controls>
        <source src="/content/drive/MyDrive/Video_classifier_example/Other test videos/football_crowds__New_Mexico_Men_s_Soccer_Crowd_Atmosphere__menslobosoccer__us9aHz.avi" type="video/mp4" style="height:300px;width:300px">
    </video>
""")
